# FastAPI Integration

**Wrap Molt-Shield as a REST API** for web application integration.

This notebook shows developers how to:
- Create a FastAPI wrapper around Molt-Shield
- Expose endpoints for sanitization and rehydration
- Handle file uploads and downloads
- Add authentication and rate limiting

## 1. Setup

In [ ]:
# ============================================================
# SETUP: Install dependencies (run this FIRST)
# ============================================================
# These packages are NOT pre-installed in Google Colab

!pip install -q lxml pydantic pyyaml fastapi uvicorn python-multipart

print("✓ Dependencies installed: lxml, pydantic, pyyaml, fastapi, uvicorn")

# Clone repository (uncomment and add your repo URL)
# !git clone https://github.com/YOUR_USERNAME/molt-shield.git /content/molt-shield

import sys
sys.path.insert(0, '/content/molt-shield/src')

import os
os.chdir('/content/molt-shield')

print("✓ Environment ready!")

## 2. The FastAPI Application

Here's the complete FastAPI application code. In production, you'd save this as `api/main.py`.

In [ ]:
"""
FastAPI application for Molt-Shield

This is the API code. In production, save as: api/main.py
Run with: uvicorn api.main:app --reload
"""

# This is conceptual code - shows the API structure

API_CODE = '''
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.responses import FileResponse
from pathlib import Path
import tempfile
import shutil

from src.gatekeeper import apply_gatekeeper
from src.config import load_config
from src.policy_engine import load_policy
from src.vault import Vault

app = FastAPI(
    title="Molt-Shield API",
    description="Zero-Trust Engineering Gateway for XML sanitization",
    version="1.0.0"
)

# Configuration
POLICY_PATH = Path("config/policy_locked.json")
CONFIG_PATH = Path("config/default.yaml")


@app.get("/")
async def root():
    """Health check endpoint."""
    return {
        "status": "healthy",
        "service": "Molt-Shield API",
        "version": "1.0.0"
    }


@app.post("/sanitize")
async def sanitize_file(
    file: UploadFile = File(...),
    policy: str = "default"
):
    """Sanitize an uploaded XML file."""
    # Validate file type
    if not file.filename.endswith('.xml'):
        raise HTTPException(400, "Only XML files are accepted")

    # Save uploaded file
    with tempfile.NamedTemporaryFile(delete=False, suffix='.xml') as tmp:
        shutil.copyfileobj(file.file, tmp)
        input_path = Path(tmp.name)

    try:
        # Load policy and config
        policy_obj = load_policy(POLICY_PATH)
        config = load_config(CONFIG_PATH)

        # Process
        sanitized_path, vault_path = apply_gatekeeper(
            input_path, policy_obj, config
        )

        # Return sanitized file
        return FileResponse(
            sanitized_path,
            media_type='application/xml',
            filename=f"sanitized_{file.filename}"
        )

    except Exception as e:
        raise HTTPException(500, f"Processing failed: {str(e)}")

    finally:
        input_path.unlink(missing_ok=True)


@app.post("/rehydrate")
async def rehydrate_file(
    file: UploadFile = File(...),
    vault_file: UploadFile = File(...)
):
    """Rehydrate a sanitized file using a vault."""
    # Save files
    # ... similar pattern to sanitize
    pass


@app.get("/health")
async def health():
    """Detailed health check."""
    return {
        "status": "healthy",
        "policy_loaded": POLICY_PATH.exists(),
        "config_loaded": CONFIG_PATH.exists()
    }
'''

print("=== FastAPI Application Structure ===")
print(API_CODE)

## 3. API Endpoints Overview

In [ ]:
# Here's what the API endpoints would look like:

ENDPOINTS = """
### Endpoints

| Method | Endpoint | Description |
|--------|----------|-------------|
| GET | `/` | Health check |
| GET | `/health` | Detailed health status |
| POST | `/sanitize` | Sanitize XML file |
| POST | `/rehydrate` | Rehydrate using vault |
| GET | `/policies` | List available policies |
| GET | `/vaults` | List available vaults |

### Example Usage

```bash
# Sanitize a file
curl -X POST -F "file=@data.xml" http://localhost:8000/sanitize -o sanitized.xml

# With Python
import requests

with open('data.xml', 'rb') as f:
    response = requests.post(
        'http://localhost:8000/sanitize',
        files={'file': f}
    )
    with open('sanitized.xml', 'wb') as out:
        out.write(response.content)
```
'''
print(ENDPOINTS)

## 4. Docker Integration

Here's how to run the API with Docker.

In [ ]:
DOCKERFILE_API = '''
# Dockerfile.api
FROM python:3.12-slim

WORKDIR /app

# Install dependencies
COPY requirements.txt .
RUN pip install --no-cache -r requirements.txt

# Copy application
COPY api/ ./api/
COPY src/ ./src/
COPY config/ ./config/

# Expose port
EXPOSE 8000

# Run with uvicorn
CMD ["uvicorn", "api.main:app", "--host", "0.0.0.0", "--port", "8000"]
'''

DOCKER_COMPOSE_API = '''
version: '3.8'

services:
  molt-shield-api:
    build:
      context: .
      dockerfile: Dockerfile.api
    ports:
      - "8000:8000"
    volumes:
      - ./data/input:/app/data/input:ro
      - ./data/output:/app/data/output
      - ./vault:/app/vault
    environment:
      - MOLT_STRICT=true
    restart: unless-stopped
'''

print("=== Dockerfile.api ===")
print(DOCKERFILE_API)
print("\n=== docker-compose.api.yml ===")
print(DOCKER_COMPOSE_API)

## 5. Complete Example Code

Here's a working example you can copy to files.

In [ ]:
# Create the directory structure
os.makedirs('api', exist_ok=True)

# Create api/__init__.py
with open('api/__init__.py', 'w') as f:
    f.write('# Molt-Shield API\n')

# Create api/main.py
MAIN_PY = '''"""Molt-Shield FastAPI Application"""
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.responses import FileResponse
from pathlib import Path
import tempfile
import shutil

app = FastAPI(
    title="Molt-Shield API",
    description="Zero-Trust Engineering Gateway for XML sanitization",
    version="1.0.0"
)

POLICY_PATH = Path("config/policy_locked.json")
CONFIG_PATH = Path("config/default.yaml")
OUTPUT_DIR = Path("data/output")
VAULT_DIR = Path("vault")


@app.get("/")
async def root():
    return {"status": "healthy", "service": "Molt-Shield API"}


@app.get("/health")
async def health():
    return {
        "status": "healthy",
        "policy_loaded": POLICY_PATH.exists(),
        "config_loaded": CONFIG_PATH.exists()
    }


@app.post("/sanitize")
async def sanitize_file(file: UploadFile = File(...)):
    if not file.filename.endswith('.xml'):
        raise HTTPException(400, "Only XML files accepted")

    # Save uploaded file
    with tempfile.NamedTemporaryFile(delete=False, suffix='.xml') as tmp:
        shutil.copyfileobj(file.file, tmp)
        input_path = Path(tmp.name)

    try:
        from src.gatekeeper import apply_gatekeeper
        from src.config import load_config
        from src.policy_engine import load_policy

        policy = load_policy(POLICY_PATH)
        config = load_config(CONFIG_PATH)

        sanitized_path, vault_path = apply_gatekeeper(
            input_path, policy, config
        )

        return FileResponse(
            sanitized_path,
            media_type='application/xml',
            filename=f"sanitized_{file.filename}"
        )
    except Exception as e:
        raise HTTPException(500, str(e))
    finally:
        input_path.unlink(missing_ok=True)
'''

with open('api/main.py', 'w') as f:
    f.write(MAIN_PY)

print("✓ Created api/main.py")

# Create requirements.txt for API
with open('requirements-api.txt', 'w') as f:
    f.write("fastapi\nuvicorn\npython-multipart\nlxml\npydantic\npyyaml\n")

print("✓ Created requirements-api.txt")

## 6. Running the API

In [ ]:
RUN_COMMANDS = '''
# === Option 1: Direct execution ===

# Install dependencies
pip install -r requirements-api.txt

# Run server
uvicorn api.main:app --host 0.0.0.0 --port 8000 --reload


# === Option 2: Docker ===

# Build image
docker build -f Dockerfile.api -t molt-shield-api .

# Run container
docker run -p 8000:8000 -v ./data:/app/data molt-shield-api


# === Option 3: Docker Compose ===

# docker-compose -f docker-compose.api.yml up
'''

print(RUN_COMMANDS)

## Summary

You now have a complete FastAPI integration:

### Created Files
- `api/__init__.py`
- `api/main.py` - FastAPI application
- `requirements-api.txt` - API dependencies

### Endpoints
- `GET /` - Health check
- `GET /health` - Detailed health
- `POST /sanitize` - Sanitize XML file

### Next Steps
- Add authentication (OAuth2, API keys)
- Add rate limiting
- Add request logging
- Add SSL/TLS
- Deploy to cloud (AWS Lambda, GCP Cloud Run, etc.)